# Tobit Regression
- 종속변수의 상한, 하한이 큰 경우 데이터 분석을 하기위한 통계적 방법

In [ ]:
import pandas as pd
import numpy as np

X = pd.read_csv('Data/train.csv')
y = X.pop('exam_score')
X = pd.get_dummies(X, drop_first=True, dtype=float)#쉬운 인코딩
X['const'] = 1

In [7]:
X

,id,age,study_hours,class_attendance,sleep_hours,gender_male,gender_other,course_b.sc,course_b.tech,course_ba,...,sleep_quality_poor,study_method_group study,study_method_mixed,study_method_online videos,study_method_self-study,facility_rating_low,facility_rating_medium,exam_difficulty_hard,exam_difficulty_moderate,const
0,0,21,7.91,98.8,4.9,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1
1,1,18,4.95,94.8,4.7,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1
2,2,20,4.68,92.6,5.8,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
3,3,19,2.00,49.5,8.3,1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,4,23,7.65,86.9,9.6,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629995,629995,18,4.86,70.7,4.1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1
629996,629996,21,7.08,54.4,4.5,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1
629997,629997,24,0.64,44.2,4.3,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1
629998,629998,20,1.54,75.1,8.2,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1


In [ ]:
from scipy.stats import norm
from statsmodels.base.model import GenericLikelihoodModel
from scipy.optimize import differential_evolution
from io import StringIO
"""
음의로그우도함수

"""
def nll(params, X, y, ymin, ymax):
    w, sigma = params[:-1], params[-1]
    z = (y-X@w)/sigma  
    ll = np.zeros_like(y)
    m = (y==ymin); ll[m] = norm.logcdf(z[m])#CDF 로거
    m = (y==ymax); ll[m] = norm.logsf(z[m])#SF 로거
    m = (y>ymin)&(y<ymax)#PDF 로거
    ll[m] = norm.logpdf(z[m])-np.log(sigma)#NLL
    return -ll 
def obj(params, X, y, ymin, ymax):
    return nll(params, X, y, ymin, ymax).sum()

class CensoredTobit(GenericLikelihoodModel):
    def __init__(self, endog, exog, ymin, ymax):
        super().__init__(endog, exog)
        self.ymin = ymin
        self.ymax = ymax

    def nloglikeobs(self, params):
        X, y, ymin, ymax = self.exog, self.endog, self.ymin, self.ymax
        return nll(params, X, y, ymin, ymax)

In [ ]:
#차등진화
from scipy.optimize import differential_evolution
bounds = [(-100, 100)] * X.shape[1]+[(5,15)] #가중치 범위와 감마 범위 설정
res0 = differential_evolution(
    obj, bounds, 
    args=(X, y, y.min(), y.max()), tol=1e-5, rng=0, 
    polish=False, updating='deferred', workers=-1)

model = CensoredTobit(y, X, ymin=y.min(), ymax=y.max())
res = model.fit(
    start_params=res0.x, 
    method='minimize',
    min_method='BFGS',
    bounds=bounds,
    maxiter=10000,
    warn_convergence=True
)

df = pd.read_csv(
    StringIO(
        res.summary(
            xname=list(X.columns)+['sigma'],
            alpha=0.05
        ).tables[1].as_csv()
    )
)
df['guess'] = [
    0,6,0.35,1.5,0,0,0,0,0,0,0,0,0,
    5,-5,-8,-5,-9,-10,-8,-4,0,0,14,10
]#범주형 데이터 처리
display(df)

Process ForkPoolWorker-9:
Process ForkPoolWorker-10:
Process ForkPoolWorker-3:
Process ForkPoolWorker-13:
Process ForkPoolWorker-7:
Process ForkPoolWorker-2:
Process ForkPoolWorker-6:
Process ForkPoolWorker-1:
Process ForkPoolWorker-12:
Process ForkPoolWorker-4:
Process ForkPoolWorker-5:
Process ForkPoolWorker-16:
Process ForkPoolWorker-8:
Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/